In [13]:
import timeit
import pyarrow
import pandas as pd
import numpy as np
import time

from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

Below we read the data from a compressed parquet file into a pandas df

In [4]:
start_time = time.time()

df = pd.read_parquet("harmonized-table.parquet.gzip")

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 304.14172101020813


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300584 entries, 0 to 14300583
Columns: 464 entries, accession to trophic_level
dtypes: object(464)
memory usage: 49.4+ GB


In [3]:
start_time = time.time()

df_describe = df.describe(include='all')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 5109.506125926971


In [4]:
df_describe

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,14300583,0,4079113,185928,13196,16138,3162,16333,12891,20305,...,999,33672,4292,2689,29968,204128,49937,1584,26878,5102
unique,14300583,0,616,9008,384,5710,500,2811,571,686,...,109,4446,875,1278,1226,5283,37670,14,1046,51
top,SAMN01127300,NaN,GRU,Unknown,Ustekinumab 45 mg,not collected,20,Missing: Not provided,missing,not applicable,...,0.3,not applicable,15.1 ÃÂµg/L,not applicable,0.25 g,9606,missing,Yes,proband,heterotroph
freq,1,NaN,1300152,20296,2907,428,418,2257,978,1547,...,146,2621,140,169,3927,92901,293,1053,4343,1057


In [5]:
df_describe.to_csv("harmonized-table.parquet_describe.tsv", sep="\t", header=True, index=True)

In [6]:
df_describe.iloc[3,].value_counts()

300      13
105      10
1         8
492       8
689       7
         ..
2388      1
56998     1
1367      1
344       1
621       1
Name: freq, Length: 347, dtype: int64

In [8]:
df_describe.loc[:, df_describe.isna().any()]

,urogenit_disord,douche,menarche,attribute,hrt
count,0,0,0,0,0
unique,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN


a first pass at values corresponding to 'no information', candidates for NA replacement for ML

In [ ]:
missing_labels = ['NaN', 'Unknown', 'not collected', 'Missing: Not provided', 'missing', 'not applicable', 'not provided', '']

In [5]:
df['env_broad_scale'].describe()

count                      1545145
unique                       17586
top       human-associated habitat
freq                        116964
Name: env_broad_scale, dtype: object

In [6]:
df['env_broad_scale'].value_counts()

human-associated habitat    116964
missing                     108895
urban biome                  86064
not applicable               72041
Bos taurus                   25803
                             ...  
AMC2-6                           1
lake environment25               1
ratGE4.4                         1
alpine grassland 116             1
Organism2                        1
Name: env_broad_scale, Length: 17586, dtype: int64

In [7]:
df['env_medium'].value_counts()

feces                     146111
faeces                    103521
soil                       99029
missing                    98177
not applicable             71522
                           ...  
131955                         1
Stool_02-047-4_6_month         1
feces_GE4.4                    1
Gut_St.Denijs_010              1
feces_21SM.w2d                 1
Name: env_medium, Length: 22520, dtype: int64

In [9]:
!pip install sklearn.preprocessing

  Could not find a version that satisfies the requirement sklearn.preprocessing (from versions: )
No matching distribution found for sklearn.preprocessing


In [12]:
#laen = preprocessing.LabelEncoder()
#df_encode = laen.fit_transform(df)

ValueError: bad input shape (14300584, 464)

In [ ]:
#onehotencoder = OneHotEncoder(categorical_features = [0])

df_encode = OneHotEncoder().fit_transform(df)

In [ ]:
df_encode.to_parquet('harmonized-table__onehot.parquet')

In [6]:
features = df.columns.drop('env_medium')
type(features)

pandas.core.indexes.base.Index

In [1]:
start_time = time.time()

y=df['env_medium']
X=df[features]

###TODO: increase splits
dtree=DecisionTreeClassifier(min_samples_split=20, random_state=9999)
#df_trim = df.drop('env_medium', axis=1) 
dtree.fit(X,y)
print('Decision Tree Classifer Created')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

NameError: name 'time' is not defined

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
visualize_tree(dt, features)

In [ ]:
start_time = time.time()

dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data, feature_names=labels,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))